In [ ]:
%pip install pandas
%pip install fastparquet
%pip install tensorflow
%pip install numpy
%pip install keras

In [3]:
import pandas as pd
import os

In [4]:
df = pd.read_parquet("data/Thermo_SRM_Pool_meta_data.parquet", engine='fastparquet')

In [5]:
print(df.shape)
print(df.head(4))

(6080606, 15)
                                       raw_file  scan_number  \
0  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34736   
1  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        25591   
2  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        24831   
3  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34870   

      modified_sequence  precursor_charge  precursor_intensity          mz  \
0  LPGSLETYVEQEQGENANDR                 2           29525630.0  1125.01950   
1       HGSLQEYLQNDTGSK                 2           13188580.0   838.89737   
2        VEEEEEINSELTAR                 2           20663460.0   824.38904   
3  LPGSLETYVEQEQGENANDR                 2           19884630.0  1125.01950   

   precursor_mz fragmentation mass_analyzer  retention_time  \
0   1125.018331           CID          ITMS          38.411   
1    838.894401           HCD          FTMS          30.394   
2    824.388401           CID          ITMS          29.737   
3   1125.018831       

In [6]:
print(df.columns)

Index(['raw_file', 'scan_number', 'modified_sequence', 'precursor_charge',
       'precursor_intensity', 'mz', 'precursor_mz', 'fragmentation',
       'mass_analyzer', 'retention_time', 'indexed_retention_time',
       'andromeda_score', 'peptide_length', 'orig_collision_energy',
       'aligned_collision_energy'],
      dtype='object')


Needed: modified_sequence, precursor charge, precursor_intensity (abundance), mz

In [7]:
for index, seq in enumerate(df["modified_sequence"]):
    if seq == "LPGSLETYVEQEQGENANDR":
        print(index, df["precursor_charge"][index], df["scan_number"][index])
        # print df row
        print("---"+ df.iloc[index].to_string())
        print("      ------------------------")

0 2 34736
---raw_file                    01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2
scan_number                                                        34736
modified_sequence                                   LPGSLETYVEQEQGENANDR
precursor_charge                                                       2
precursor_intensity                                           29525630.0
mz                                                             1125.0195
precursor_mz                                                 1125.018331
fragmentation                                                        CID
mass_analyzer                                                       ITMS
retention_time                                                    38.411
indexed_retention_time                                         62.827337
andromeda_score                                                   447.14
peptide_length                                                        20
orig_collision_energy                 

Test: https://www.kirenz.com/post/2022-06-17-deep-learning-and-keras-data-preprocessing-with-structured-data/

In [8]:
#charge states 1-5
y_label = "precursor_charge"

In [9]:
# Convert to categorical

# make a list of all categorical variables
cat_convert = ['scan_number', 'precursor_charge', 'precursor_intensity']

# convert variables
for i in cat_convert:
    df[i] = df[i].astype("category")

In [10]:
# Make list of all numerical data (except label)
list_num = df.drop(columns=[y_label]).select_dtypes(include=[np.number]).columns.tolist()

# Make list of all categorical data which is stored as integers (except label)
list_cat_int = df.drop(columns=[y_label]).select_dtypes(include=['category']).columns.tolist()

# Make list of all categorical data which is stored as string (except label)
list_cat_string = df.drop(columns=[y_label]).select_dtypes(include=['string']).columns.tolist()

NameError: name 'np' is not defined

In [ ]:
df.info()

In [ ]:
# Make validation data
df_val = df.sample(frac=0.2, random_state=1337)

# Create training data
df_train = df.drop(df_val.index)

In [ ]:
# Save training data
df_train.to_csv("data/df_train.csv", index=False)

In [ ]:
print(
    "Using %d samples for training and %d for validation"
    % (len(df_train), len(df_val))
)

In [ ]:
# Define a function to create our tensors

def dataframe_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop(y_label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    df = ds.prefetch(batch_size)
    return ds

In [ ]:
import tensorflow as tf
batch_size = 32

ds_train = dataframe_to_dataset(df_train, shuffle=True, batch_size=batch_size)
ds_val = dataframe_to_dataset(df_val, shuffle=True, batch_size=batch_size)

In [ ]:
# Define numerical preprocessing function
def get_normalization_layer(name, dataset):

    # Create a Normalization layer for our feature
    normalizer = layers.Normalization(axis=None)

    # Prepare a dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    return normalizer

In [11]:
from tensorflow.keras import layers
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):

  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

ModuleNotFoundError: No module named 'tensorflow'

In [52]:
all_inputs = []
encoded_features = []

In [54]:
# Numerical features
for feature in list_num:
  numeric_feature = tf.keras.Input(shape=(1,), name=feature)
  normalization_layer = get_normalization_layer(feature, ds_train)
  encoded_numeric_feature = normalization_layer(numeric_feature)
  all_inputs.append(numeric_feature)
  encoded_features.append(encoded_numeric_feature)

KeyboardInterrupt: 

In [ ]:
for feature in list_cat_int:
  categorical_feature = tf.keras.Input(shape=(1,), name=feature, dtype='int32')
  encoding_layer = get_category_encoding_layer(name=feature,
                                               dataset=ds_train,
                                               dtype='int32',
                                               max_tokens=5)
  encoded_categorical_feature = encoding_layer(categorical_feature)
  all_inputs.append(categorical_feature)
  encoded_features.append(encoded_categorical_feature)

In [ ]:
for feature in list_cat_string:
  categorical_feature = tf.keras.Input(shape=(1,), name=feature, dtype='string')
  encoding_layer = get_category_encoding_layer(name=feature,
                                               dataset=ds_train,
                                               dtype='string',
                                               max_tokens=5)
  encoded_categorical_feature = encoding_layer(categorical_feature)
  all_inputs.append(categorical_feature)
  encoded_features.append(encoded_categorical_feature)

In [ ]:
# Input
all_features = layers.concatenate(encoded_features)

# First layer
x = layers.Dense(32, activation="relu")(all_features)

# Dropout to prevent overvitting
x = layers.Dropout(0.5)(x)

# Output layer
output = layers.Dense(1, activation="sigmoid")(x)

# Group all layers
model = tf.keras.Model(all_inputs, output)

In [ ]:
model.compile(optimizer="adam",
              loss ="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
# `rankdir='LR'` is to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
%%time
model.fit(ds_train, epochs=10, validation_data=ds_val)

In [ ]:
loss, accuracy = model.evaluate(ds_val)

print("Accuracy", round(accuracy, 2))

Inference

In [ ]:
model.save('data/my_hd_classifier')

In [ ]:
reloaded_model = tf.keras.models.load_model('my_hd_classifier')

In [ ]:
sample = {
    "modified_sequence": LPGSLETYVEQEQGENANDR,
    "precursor_charge": 1,
    "precursor_intensity": 1312312,
}

In [ ]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [ ]:
predictions = reloaded_model.predict(input_dict)

In [ ]:
print(
    "This sequence a %.1f percent probability "
    "of having the given charge state, as evaluated by our model." % (100 * predictions[0][0],)
)